In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data_path = 'logistic_regression_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,0,worldcom boss left books alone former worldc...
1,1,tigers wary of farrell gamble leicester say ...
2,1,yeading face newcastle in fa cup premiership s...
3,1,henman hopes ended in dubai third seed tim hen...
4,1,wilkinson fit to face edinburgh england captai...
...,...,...
1012,0,wall street cool to ebay s profit shares in on...
1013,0,ban on forced retirement under 65 employers wi...
1014,1,time to get tough on friendlies for an intern...
1015,0,christmas shoppers flock to tills shops all ov...


In [6]:



text = df['text']
category = df['category']
train_text, test_text, train_category, test_category = train_test_split(text, category, random_state=42, stratify=category, test_size=0.01)

print("Shape of Train_Text = ", train_text.shape)
print("Shape of Test_Text = ", test_text.shape)
print("Shape of Train_Category = ", train_category.shape)
print("Shape of Test_Category = ", test_category.shape)
print(test_category.shape[0])
test_category = np.array(test_category).reshape(test_category.shape[0],1)
train_category = np.array(train_category).reshape(train_category.shape[0],1)
print(test_category.shape,train_category.shape)

Shape of Train_Text =  (1006,)
Shape of Test_Text =  (11,)
Shape of Train_Category =  (1006,)
Shape of Test_Category =  (11,)
11
(11, 1) (1006, 1)


In [7]:
test_category = np.where(test_category<1,-1,1)
train_category = np.where(train_category<1,-1,1)
train_category


array([[-1],
       [ 1],
       [-1],
       ...,
       [-1],
       [-1],
       [ 1]])

In [8]:
tf_idf_vect = TfidfVectorizer(max_features=2000, min_df=10,ngram_range=(2,3))
tf_idf_fit = tf_idf_vect.fit(train_text)
tf_idf_transform = tf_idf_fit.transform(train_text)


In [9]:
type(tf_idf_transform)#.shape

scipy.sparse.csr.csr_matrix

In [10]:
from sklearn.preprocessing import StandardScaler#(*, copy=True, with_mean=True, with_std=True)[source]¶

In [11]:
tf_idf_dense = tf_idf_transform.todense()
scaler = StandardScaler()
prep_fit_train = scaler.fit(tf_idf_dense)
trans_train_prep = scaler.transform(tf_idf_dense)

In [12]:
trans_train_prep
#note : instead of using dense, we can still do some hack.
#in a column , all the zeroes will have same value -mu/sigma
#the changes for the non-zero values in the original sparse matrix can be saved in a new sparse matrix and a list with size
#equal to number of columns can be used to save corresponding -mu/sigma of each column

array([[-0.13799001, -0.1456903 , -0.09874225, ..., -0.18648737,
        -0.17989714, -0.14182251],
       [-0.13799001, -0.1456903 , -0.09874225, ..., -0.18648737,
        -0.17989714, -0.14182251],
       [-0.13799001, -0.1456903 , -0.09874225, ..., -0.18648737,
        -0.17989714, -0.14182251],
       ...,
       [ 3.97076202, -0.1456903 , -0.09874225, ..., -0.18648737,
        -0.17989714, -0.14182251],
       [-0.13799001, -0.1456903 , -0.09874225, ..., -0.18648737,
        -0.17989714, -0.14182251],
       [-0.13799001, -0.1456903 , -0.09874225, ..., -0.18648737,
        -0.17989714, -0.14182251]])

In [13]:
trans_train_prep.shape

(1006, 2000)

In [14]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [15]:
def log_reg(X_train,Y_train,learning_rate,iterations,lamb):
    X= X_train
    Y= Y_train
    m = X.shape[0]  # m is number of data-points
    n = X.shape[1] #n is number of features
    #x shape is m*n and w shape is n*1
    
    W = np.zeros((n,1))  #W basically consists weights of each feature. So,it's a 2d matrix of n*1 size
    D = 0                     # D is scalar but won't harm because of property of numpy 
    cost_lst = []
    for i in range(iterations):     
        #Z =np.dot(Y.T,(np.dot(X,W) + D)) #applying signs to respective distances is not vector in nature. 
        # Z will be a m*1 matrix where each row represents a data-point ,and the element of the row is the distance of that datapoint
        Z=Y*(np.dot(X,W) + D)
        A =sigmoid(Z)             #from the plane
        cost_each = -np.log(A+1e-10) +lamb*n # using math.log will expect scalar and passing np array will lead to this error "only size-1 arrays can be converted to Python scalars"
        cost = np.sum(cost_each)                                                      #https://stackoverflow.com/questions/36680402/typeerror-only-length-1-arrays-can-be-converted-to-python-scalars-while-plot-sh/42350817
        dl_mid = -(1/m)*Y*(1-A) # this gives m*1 matrix.
        dl_by_dw = np.dot(X.T,dl_mid)
        #dl_by_dD = -(1/m)*(np.exp(-Z)/1+np.exp(-Z))*Y #this gives m*1 matrix which is totally consistent with nature of D
        
        
        W = W - learning_rate*dl_by_dw -lamb
        D = D - learning_rate*np.sum(dl_mid) -lamb
        
        cost_lst.append(cost)
        
        if(i%1000==0):
            print(cost)
    return W,D,cost_lst
        

In [16]:
iterations = 10000
lamb =0.00000001
learning_rate = 0.0005
X_train = trans_train_prep
Y_train = train_category

W,D,cost_lst = log_reg(X_train,Y_train,learning_rate,iterations,lamb)

697.3261834421048
133.3089879988409
76.45441225269414
54.123619467131164
41.994501221757325
34.33408352141688
29.046506827672587
25.17418735517442
22.215081139495453
19.879922950836466


In [17]:
cost_lst[9999]
#print(W.shape,test_X.shape)


17.99180140380644

def random_model(X,Y):
    m = X.shape[0] # number of points
    n = X.shape[1] # number of dimensions
    Y_rand = np.random.uniform(low=-200.0, high = 200.0,size = m) #generating some distances. I think the range is justified as beyond this range the rate of change in the curve of sigma(which is goinng to be used for probabilities) very much dies
    Z = Y*Y_rand # these are signed distances
    A = sigmoid(Z)
    cost_each = -np.log(A+1e-10)
    cost = np.sum(cost_each)
    
    return cost
random_model(X,Y)

In [20]:
probs_lst = []
for i in range(trans_train_prep.shape[0]):
    rand_prob= np.random.rand(1)
    probs_lst.append(rand_prob)
label_lst =list(map(lambda x: 1 if x>=0.5 else 0, probs_lst))
probs_lst = np.array(probs_lst)
label_lst = np.array(label_lst)

probs_lst_mod = list(map(lambda x: x if x>=0.5 else 1-x , probs_lst))
probs_lst_mod = np.array(probs_lst_mod)
log_loss = (np.log(probs_lst_mod).sum())/trans_train_prep.shape[0]


print("Log loss on Train Data using Random Model",log_loss)

Log loss on Train Data using Random Model -0.29703711701756247


In [26]:
new_probs = np.random.uniform(0,1,trans_train_prep.shape[0])
print(new_probs)
new_probs = list(map(lambda x:x if x>=0.5 else 1-x , new_probs))
new_probs = np.array(new_probs)
print("newest prob are",new_probs)

print("log_loss is", (np.log(new_probs).sum()))

[0.98794286 0.90133477 0.77822173 ... 0.21975476 0.94837135 0.4104433 ]
newest prob are [0.98794286 0.90133477 0.77822173 ... 0.78024524 0.94837135 0.5895567 ]
log_loss is -296.96899968725484


In [63]:
def predict_label(test_X,test_Y):
    z = np.dot(test_X,W) + D
    res = sigmoid(z)
    labels = np.where(res<0.5,-1,1)
    
    acc = 1-(np.sum(np.absolute(test_Y-labels))/test_Y.shape[0])
    
    return acc,labels
       
    
    

In [64]:
#train_accuracy
acc_train , label_train = predict_label(X_train,Y_train)
acc_train

1.0

In [65]:
X_test = tf_idf_fit.transform(test_text).todense()
#X_test =trans_train_prep #scaler.fit_transform(X)
Y_test = test_category
acc_test , label_test = predict_label(X_test,Y_test)
acc_test

1.0

In [37]:
print(W.shape,X_test.shape)

(2000, 1) (11, 2000)


In [68]:
np.count_nonzero(W)

2000